# You know what's on for dinner?

Let's see how OSM and Yelp make that choice a little easier:

- Find restaurants in 10 min walking radius from SOTM location
- Filter restaurants with Yelp rating >= 4
- Display in a map and set metadata to popups

In [ ]:
import folium
from folium.plugins import MeasureControl
from shapely import geometry
from IPython.display import HTML

from pprint import pprint

# Set up *openrouteservice* client

1. Sign up on our [homepage](https://openrouteservice.org/sign-up/) 
2. Create API key on our [developer portal](https://openrouteservice.org/dev/#/home)
3. Review our [ToS](https://openrouteservice.org/terms-of-service/)
4. `pip install openrouteservice`
5. Fire it up!

Here we will:
- _**Geocode**_ the SOTM official address
- Create an _**isochrone**_ with 10 mins walking radius

In [35]:
import openrouteservice as ors

clnt = ors.Client(key='58d904a497c67e00015b45fc3535fd17dd8948bb8cb330e216534497')

params_geocode = {'text': 'Politecnico di Milano ‐ Piazza Leonardo da Vinci, Milan, Lombardy, Italy'}

sotm_location = clnt.pelias_search(**params_geocode)['features'] # Geocode from official address
sotm_isochrone = clnt.isochrones(locations=sotm_location[0]['geometry']['coordinates'], # Calculate isochrones
                                 intervals=[600],
                                 segments=600,
                                 profile='foot-walking')

In [ ]:
x, y = sotm_location[0]['geometry']['coordinates']
def createBaseMap(width, height, include=True):
    bar_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/bar.png',
                                       icon_size=(30, 30))
    sotm_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/sotm_map_marker.png', icon_size=(58, 70))

    fig = folium.Figure(width=width, height=height)
    m = folium.Map(tiles='stamenwatercolor',location=(y, x), zoom_start=15)
    folium.map.Marker([y, x], icon=sotm_icon).add_to(m)
    if include:
        folium.features.GeoJson(sotm_isochrone).add_to(m)
    
    m.add_child(MeasureControl())
    
    return fig, m

## Set up a basic `folium` map
A little magic is hidden here.

In [36]:
fig, m = createBaseMap('100%', 600) # Call map factory with figure dimensions
# folium.map.Marker([y, x], icon=sotm_icon).add_to(m)
# folium.features.GeoJson(sotm_isochrone).add_to(m)
fig.add_child(m)
fig

## Request restaurant POI's

- `/pois?` endpoint takes GeoJSON
- we can pipe the GeoJSON output of `/isochrones?` directly to `/pois?` to limit the query to the isochrones geometry

In [37]:
param_pois = {'request': 'pois',
            'geojson': sotm_isochrone['features'][0]['geometry'],
            'filter_category_ids': [570], # ID list: https://github.com/GIScience/openrouteservice-docs#sustenance--560
            'sortby': 'distance'}

sotm_restaurants = clnt.places(**param_pois)['features']
display(HTML('<br><br>Amount of restaurants within 10 mins walking radius: <strong>{}</strong>'.format(len(sotm_restaurants))))

## Yelp rating

- use the [Yelp `businesses` API](https://www.yelp.de/developers/documentation/v3/business) to find out about the reviews
- extract `name`, `rating` and `url` to feature popup

In [ ]:
from urllib.parse import urlencode
import json
import requests

yelp_url = 'https://api.yelp.com/v3/businesses/search?' # Create a Yelp account and a API key: https://www.yelp.com/developers
yelp_header = json.load(open('token.json')) # use secret from file: {'Authorization': 'Bearer MyAPIkey'}
yelp_params = {'limit': 1}

def yelp_response(**kwargs):
    yelp_params.update({'term': r_name,
                        'longitude': r_coords[0],
                        'latitude': r_coords[1]})
    # Concat params to URL encoded string
    yelp_url_params = requests.utils.unquote_unreserved(urlencode(list(yelp_params.items())))
    try:
        yelp_response = requests.get(yelp_url + yelp_url_params,
                                     headers=yelp_header).json()['businesses'][0]
    except:
        return False

    if yelp_response['rating'] < 4.0:
        return False
    
    return yelp_response
    

In [38]:
import branca

fig, m = createBaseMap('100%', '500')
for idx, restaurant in enumerate(sotm_restaurants):
    r_name = restaurant['properties']['osm_tags'].get('name', '')
    if r_name != '':
        r_coords = restaurant['geometry']['coordinates']
        yelp_return = yelp_response(r_name=r_name, # Call hidden function to retrieve Yelp results with rating >= 4 stars
                                    r_coords=r_coords)
        if yelp_return:
            restaurant['yelp'] = yelp_return
            restaurant_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/restaurant2.png',
                                                   icon_size=(50, 50))
            popup_html = '<h4>{0}</h4><strong><a href="{1}" target="_blank">URL</a><br>Stars: </strong>{2}'.format(r_name, yelp_return['url'], yelp_return['rating'])
            iframe = branca.element.IFrame(html=popup_html, width=300, height=100)
            popup = folium.Popup(iframe, max_width=300)
            folium.map.Marker(list(reversed(r_coords)), icon=restaurant_icon, popup=popup).add_to(m)
fig.add_child(m)
fig

In [ ]:
from ipywidgets import widgets
from ipywidgets import interact, interactive
from IPython.display import display, clear_output

# Define change function for drop-down
def on_category_change(change):
    # First we need to clear_output() entirely and then re-draw all components
    clear_output()
    
    x, y = sotm_location[0]['geometry']['coordinates']
    fig2, map2 = createBaseMap('100%', 600, include=False)
    
    for restaurant in sotm_restaurants:
        if restaurant.get('yelp', '') != '':
            if restaurant['yelp']['categories'][0]['title'] == change['new']:
                restaurant_coords = restaurant['geometry']['coordinates']
                clnt = ors.Client(key='58d904a497c67e00015b45fc3535fd17dd8948bb8cb330e216534497')
                route = clnt.directions(coordinates=[[x,y],
                                                     restaurant_coords
                                                    ],
                                       format_out='geojson',
                                       profile='foot-walking',
                                       geometry_format='geojson')
                
                route_properties = route['features'][0]['properties']
                folium.features.GeoJson(route).add_to(map2)

                restaurant_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/restaurant2.png',
                                                             icon_size=(50, 50))
                popup_html = """<h4>{0}</h4>
                                <strong><a href="{1}" target="_blank">URL</a></strong><br>
                                <strong>Category: </strong>{2}<br>
                                <strong>Walking time: </strong>{3:.2f} mins<br>
                                <strong>Reviews: </strong>{4}<br>                                
                                <strong>Stars: </strong>{5}""".format(restaurant['yelp']['name'],
                                                              yelp_return['url'],
                                                              change['new'],
                                                              route_properties['summary'][0]['duration'] / 60,
                                                              restaurant['yelp']['review_count'],
                                                              yelp_return['rating'])
                
                iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
                popup = folium.Popup(iframe, max_width=300)
                folium.map.Marker(list(reversed(restaurant_coords)),
                                  icon=restaurant_icon,
                                  popup=popup,
                                 ).add_to(map2)
    fig2.add_child(map2)
    # Now we re-add the widgets to the cleared output area
    display(widget_dd)
    display(fig2)

In [39]:
category_titles = []
for restaurant in sotm_restaurants:
    if restaurant.get('yelp', '') != '':
        category_titles.append(restaurant['yelp']['categories'][0]['title'])

widget_dd = widgets.Dropdown(options=set(category_titles))
widget_dd.observe(on_category_change, names='value')
display(widget_dd)

Dropdown(index=3, options=('Lumbard', 'Pizza', 'Lounges', 'Korean', 'African', 'Italian', 'Asian Fusion', 'Con…